In [1]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import seaborn as sns
from scipy.signal import butter,filtfilt,welch
from scipy import signal
import optuna
import numpy as np  
import optuna.visualization as vis
import plotly

/usr/lib/python3/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
low_cutoff = 0.01  # Low cut-off frequency (Hz)
high_cutoff = 80.0  # High cut-off frequency (Hz)
order = 4  # Order of the filter
sampling_rate_equipo= 500 
b, a = signal.butter(order, [low_cutoff / (0.5 * sampling_rate_equipo), high_cutoff / (0.5 * sampling_rate_equipo)], btype='band')

In [3]:

fs = 500  # Sampling rate
nperseg = 256  # Segment length
noverlap = 128 # Overlap between segments
sampling_rate = 2000 ### simulacion 

resultado_compilacion = subprocess.run("g++ -fopenmp -O3 main.cpp Matrix_conect.cpp -o main_multi", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
if resultado_compilacion.returncode != 0:
    print("Error durante la compilación:")
    print(resultado_compilacion.stderr)

def Simulado_EGG(params):
    L, P, inhibidoras, trest, trelative, alpha, tmax, type_matrix = params
        
    # Ejecución del programa
    resultado_ejecucion = subprocess.run(f"./main_multi {L} {P} {inhibidoras} {trest} {trelative} {alpha} {tmax} {type_matrix}", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if resultado_ejecucion.returncode != 0:
        print("Error durante la ejecución:")
        print(resultado_ejecucion.stderr)
        return None
    
    # Procesamiento de los datos
    lines = resultado_ejecucion.stdout.strip().split('\n')
    data_list = [[float(num) for num in line.split()] for line in lines]
    data = np.array(data_list)[30:,1]

    data=(data-np.mean(data)) /np.std(data)
    
    n_samples_resampled = int(len(data) * (fs/sampling_rate))
    data_resampled = signal.resample(data, n_samples_resampled)

    butter = signal.filtfilt(b, a,data_resampled)

    frequencies, psd = signal.welch(butter, fs, nperseg=nperseg, noverlap=noverlap)
    frequencies=np.array(frequencies)


    welch_data = np.array([10 * np.log10(psd[i]+ 1e-10) for i in range(len(frequencies))])
    
    delta = welch_data[np.logical_and(frequencies >= 0, frequencies <= 3)]
    theta = welch_data[np.logical_and(frequencies >= 3, frequencies <= 7)]
    alpha = welch_data[np.logical_and(frequencies >= 7, frequencies <= 14)]
    beta = welch_data[np.logical_and(frequencies >= 14, frequencies <= 30)]

    return {
        'data': data,"data_resample":data_resampled,"butter": butter,
        "frequencies": frequencies,"welch_data":welch_data,
        'delta':[delta.mean(),delta.std()],
        'theta':[theta.mean(), theta.std()],
        'alpha':[ alpha.mean(),alpha.std()],
        'beta': [beta.mean(), beta.std()]
    }



# Datos Reales

 Tanto como los datos reales como lo simulados pueden ser normlaizados(por su maximo), los valores real estan en microvoltios, en la simulacion estan con 0.00070 microvoltios reposo 

In [4]:
def Datos_reales(nombres):
    Datos = np.empty(len(nombres), dtype=object)
    welch_datos = np.empty(len(nombres), dtype=object)
    Frequencies = np.empty(len(nombres), dtype=object)
    Delta = np.empty(len(nombres), dtype=object)
    Theta = np.empty(len(nombres), dtype=object)
    Alpha = np.empty(len(nombres), dtype=object)
    Beta = np.empty(len(nombres), dtype=object)
    Nombres=np.empty(len(nombres), dtype="U100")

    # Inicialización de arrays para promedios de bandas
    main_Delta = np.zeros(2)
    main_Theta = np.zeros(2)
    main_Alpha = np.zeros(2)
    main_Beta = np.zeros(2)
    for i, nombre in enumerate(nombres):  
        Nombres[i]=nombre 



        data = np.loadtxt(nombre)
        data=(data-np.mean(data)) /np.std(data)
        Datos[i]=data


        frequencies, psd = signal.welch(data, fs, nperseg=nperseg, noverlap=noverlap)
        Frequencies[i]=frequencies=np.array(frequencies)

        welch_data = np.array([10 * np.log10(psd[i]+ 1e-10) for i in range(len(frequencies))])
        welch_datos[i]=welch_data

        delta = welch_data[np.logical_and(frequencies >= 1, frequencies <= 3)]
        theta = welch_data[np.logical_and(frequencies >= 4, frequencies <= 7)]
        alpha = welch_data[np.logical_and(frequencies >= 7, frequencies <= 13)]
        beta = welch_data[np.logical_and(frequencies >= 14, frequencies <= 30)]

        Delta[i]=[delta.mean(),delta.std()]
        Theta[i]=[theta.mean(),theta.std()]
        Alpha[i]=[alpha.mean(),alpha.std()]
        Beta[i]=[beta.mean(),beta.std()]

        main_Delta += np.array([delta.mean(), delta.std()]) / len(nombres)
        main_Theta += np.array([theta.mean(), theta.std()]) / len(nombres)
        main_Alpha += np.array([alpha.mean(), alpha.std()]) / len(nombres)
        main_Beta += np.array([beta.mean(), beta.std()]) / len(nombres)

    return {
        "Nombres":Nombres,
        'datos': Datos, "frequencies": Frequencies,  "welch_data":welch_datos,
        'delta':Delta,'theta':Theta,'alpha':Alpha,'beta':Beta,
        'main_Delta': main_Delta,'main_Theta':main_Theta,"main_Alpha":main_Alpha,"main_Beta":main_Beta
    }


In [5]:
Params={'L': 63, 'P': 0.47531114678430786, 'Inh': 0.5245008243721399, 'Trest': -6.325934192545346, 'Trelative': 2.325451423214342, 'C_h': 9.485832803300346, 'Tipo_red': 1}

initial_params = [Params["L"], Params["P"],Params["Inh"], Params["Trest"],Params["Trelative"], Params["C_h"] ,8030, Params["Tipo_red"]]
generated_data = Simulado_EGG(initial_params)
reales_total = Datos_reales(["P3_MNE.txt","Pz_MNE.txt","P4_MNE.txt"])

In [6]:
fig, ax = plt.subplots(figsize=[12, 5])

print(reales_total["Nombres"])
#ax.plot(generated_data["data_resample"],label="Simulado")
ax.plot(generated_data["butter"],label="Simulado con filtro butter")
for i in range(3):
    ax.plot(reales_total["datos"][i],"-.", label=reales_total["Nombres"][i],alpha=0.8)


plt.xlabel("Pasos de 0.002 segundos")
plt.ylabel(r"$\mu V$")
plt.legend()  # Esto añade una leyenda para identificar cada línea
plt.grid()
plt.show()  # Mostrar todos los gráficos en la misma figura


['P3_MNE.txt' 'Pz_MNE.txt' 'P4_MNE.txt']


In [7]:
fig, ax = plt.subplots(figsize=[12, 5])

print(reales_total["Nombres"])
print("Delta",generated_data["delta"],"----",reales_total["main_Delta"])
print("Theta",generated_data["theta"],"----",reales_total["main_Theta"])
print("Alpha",generated_data["alpha"],"----",reales_total["main_Alpha"])
print("Beta",generated_data["beta"],"----",reales_total["main_Beta"])

ax.plot(generated_data["frequencies"],generated_data["welch_data"],label="Simulacion")
for i in range(3):
    ax.plot(reales_total["frequencies"][i],reales_total["welch_data"][i], label=reales_total["Nombres"][i])

plt.xlabel("Frecuencia")
plt.ylabel("psd dB ")
plt.grid()
plt.legend()  # Esto añade una leyenda para identificar cada línea
plt.show()  # Mostrar todos los gráficos en la misma figura


['P3_MNE.txt' 'Pz_MNE.txt' 'P4_MNE.txt']
Delta [-43.713753902164484, 0.26287500670357034] ---- [-11.74661973   0.        ]
Theta [-64.27995611638023, 3.672805897092644] ---- [-15.54270109   0.        ]
Alpha [-68.57409525419885, 1.8737586378343927] ---- [-17.20621336   1.28765258]
Beta [-82.06173701513575, 5.43465060196636] ---- [-24.64981692   1.54922788]


In [10]:


def objective(trial):
    params = np.zeros(8)
    params[0] = trial.suggest_int('L', 10, 60)
    params[1] = trial.suggest_float('P', 0.0, 1.0)
    params[2] = trial.suggest_float('Inh', 0.0, 1.0)
    params[3] = trial.suggest_float('Trest', -10.0, 10.0)
    params[4] = trial.suggest_float('Trelative', params[3], params[3] + 10)
    params[5] = trial.suggest_float('C_h', 0, 10.0)
    params[6] = 8030
    params[7] = trial.suggest_int('Tipo_red', 0, 4)

    generated_data_total = Simulado_EGG(params)

    # Asegúrate de que no haya NaN en las diferencias
    if generated_data_total["delta"][1] - generated_data_total["delta"][1] == 0:
        dif_Delta = generated_data_total["delta"] - reales_total["main_Delta"]
        dif_Theta = generated_data_total["theta"] - reales_total["main_Theta"]
        dif_Alpha = generated_data_total["alpha"] - reales_total["main_Alpha"]
        dif_Beta = generated_data_total["beta"] - reales_total["main_Beta"]

        # Calcular las diferencias absolutas
        abs_dif_Delta = np.sum(np.abs(dif_Delta))
        abs_dif_Theta = np.sum(np.abs(dif_Theta))
        abs_dif_Alpha = np.sum(np.abs(dif_Alpha))
        abs_dif_Beta = np.sum(np.abs(dif_Beta))      

        # Retornar como una tupla
        return abs_dif_Delta, abs_dif_Theta, abs_dif_Alpha, abs_dif_Beta
         
    else:
        return float('inf'), float('inf'), float('inf'), float('inf')

# Crear y optimizar el estudio para múltiples objetivos
study = optuna.create_study(directions=["minimize", "minimize", "minimize", "minimize"],
                             study_name='4bandas_full_Normal_Z_score',
                             storage='sqlite:///Estudios_multi/4bandas_full_Normal_Z_score.db',
                             load_if_exists=True)

# Continúa con la optimización del estudio
study.optimize(objective, n_trials=500)


[I 2024-10-04 11:52:02,729] Using an existing study with name '4bandas_full_Normal_Z_score' instead of creating a new one.


[I 2024-10-04 11:52:04,623] Trial 45 finished with values: [88.25336769633947, 84.4572934935002, 84.08143858709748, 76.8994109577886] and parameters: {'L': 27, 'P': 0.5209179894972651, 'Inh': 0.11011220631330787, 'Trest': 9.183403523340957, 'Trelative': 10.751468082439812, 'C_h': 3.275833653082598, 'Tipo_red': 1}. 
[I 2024-10-04 11:52:08,202] Trial 46 finished with values: [36.1851371829722, 58.44949524756188, 61.85838183995941, 61.474683448624994] and parameters: {'L': 36, 'P': 0.8472362644873268, 'Inh': 0.11414784120657528, 'Trest': -1.4394444572133978, 'Trelative': 1.7638179858127216, 'C_h': 2.0058677968479866, 'Tipo_red': 1}. 
[I 2024-10-04 11:52:11,692] Trial 47 finished with values: [57.346155143574705, 52.18626990281553, 70.39629910512312, 73.53089797214768] and parameters: {'L': 34, 'P': 0.6321717996155796, 'Inh': 0.11546294763435005, 'Trest': -6.493870401640733, 'Trelative': -6.383204156231269, 'C_h': 6.733821818090858, 'Tipo_red': 3}. 
[I 2024-10-04 11:53:58,745] Trial 48 fin

KeyboardInterrupt: 

In [9]:
# Imprimir el nombre del estudio
print(study.study_name)

# Filtrar los trials válidos
valid_trials = [t for t in study.trials if t.values is not None]

# Obtener los mejores trials en función de uno de los objetivos, por ejemplo, el primero
top_trials = sorted(valid_trials, key=lambda t: t.values[0])[:10]

# Imprimir los mejores trials
for i, trial in enumerate(top_trials):
    print(f"Trial {i}:")
    print(f"  Params: {trial.params}")
    print(f"  Values: {trial.values}")  # Esto ahora devolverá una lista de valores


4bandas_full_Normal_Z_score
Trial 0:
  Params: {'L': 16, 'P': 0.30226772281920133, 'Inh': 0.03109138799049993, 'Trest': 0.615365897544125, 'Trelative': 0.9754589792702744, 'C_h': 7.468917950971122, 'Tipo_red': 2}
  Values: [9.732036807398394, 26.630212656189222, 39.67186636967958, 36.95108135612775]
Trial 1:
  Params: {'L': 68, 'P': 0.8830787316541423, 'Inh': 0.3173657341587369, 'Trest': -0.1143972086075884, 'Trelative': 4.7780744334247345, 'C_h': 3.1352040214364605, 'Tipo_red': 1}
  Values: [10.51308176159979, 5.6972830867530995, 8.175797906958234, 7.628548938027679]
Trial 2:
  Params: {'L': 87, 'P': 0.1848250019744816, 'Inh': 0.4012727193049591, 'Trest': -3.990719935590345, 'Trelative': -1.200526308674989, 'C_h': 2.500638194156375, 'Tipo_red': 2}
  Values: [11.49659118124435, 22.418208251167343, 45.304829147910084, 48.81125046431635]
Trial 3:
  Params: {'L': 93, 'P': 0.8898638965350665, 'Inh': 0.8680139373257085, 'Trest': -7.711073265849667, 'Trelative': 0.6555299991724706, 'C_h': 3.

In [29]:
vis.plot_param_importances(study)

In [30]:
target = lambda trial: trial.values[0]  # Primer objetivo a visualizar

# Ahora puedes graficar el gráfico de coordenadas paralelas
vis.plot_parallel_coordinate(study, target=target, target_name='Primer Objetivo')

In [31]:
target = lambda trial: trial.values[0]  # Primer objetivo a visualizar

# Graficar el gráfico de rebanada
vis.plot_slice(study, target=target, target_name='Primer Objetivo')

In [26]:
# Graficar para el segundo objetivo
target_second = lambda trial: trial.values[1]  # Segundo objetivo
vis.plot_slice(study, target=target_second, target_name='Segundo Objetivo')
